# Data Split
This notebook partitions the merged dataset into distinct subsets for model development, test, validation, and production.

## Goals
1. **Implement Deterministic Splitting**: Ensure the split is fully reproducible across reruns by using a stable sort order rather than random sampling.
2. **Enforce Stratification**: Use stratified bucketing across both `source_dataset` and `label` to ensure the massive size of the TON_IoT dataset does not overwhelm smaller sources like UNSW-NB15 in any specific split.
3. **Isolate Holdout Data**: Maintain the integrity of validation, test, and production sets by ensuring they remain representative of the original data, while confining any future rebalancing or sampling strictly to the training set.

In [51]:
!pip -q install "PyAthena[SQLAlchemy]" sqlalchemy s3fs

In [52]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

## Connect to Athena

In [53]:
sess = sagemaker.Session()
region = boto3.Session().region_name

results_bucket = sess.default_bucket()
athena_results_path = f"s3://{results_bucket}/athena/staging/"

database_name = "aai540_eda"

engine = create_engine(
    f"awsathena+rest://@athena.{region}.amazonaws.com:443/{database_name}",
    connect_args={"s3_staging_dir": athena_results_path, "region_name": region},
)
print("Region:", region)
print("Athena results:", athena_results_path)

Region: us-east-1
Athena results: s3://sagemaker-us-east-1-933747558592/athena/staging/


In [54]:
# Helper functions for queries
def exec_ddl(sql: str):
    with engine.begin() as conn:
        conn.execute(text(sql))

def read_sql(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, engine)

## Quick EDA before splitting merged dataset

### Explore a few merged dataset rows

In [55]:
read_sql(f"""
SELECT *
FROM {database_name}.feature_engineered_cleaned
LIMIT 5
""")

,duration,pkt_total,bytes_total,pkt_fwd,pkt_bwd,bytes_fwd,bytes_bwd,label,original_attack_type,attack_category,source_dataset,pkt_rate,byte_rate,bytes_per_pkt,pkt_ratio,byte_ratio
0,4.999909,3530,0,3530,0,0,0,1,ddos,DoS/DDoS,TON_IoT,706.012849,0.0,0.0,3530.0,0.0
1,61.819354,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.080881,0.0,0.0,1.0,0.0
2,60.855999,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.082161,0.0,0.0,1.0,0.0
3,60.859135,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.082157,0.0,0.0,1.0,0.0
4,60.860039,5,0,3,2,0,0,1,ddos,DoS/DDoS,TON_IoT,0.082156,0.0,0.0,1.0,0.0


### Data source distribution in the merged dataset

In [56]:
rows_per_source = read_sql(f"""
SELECT
  source_dataset,
  COUNT(*) AS row_count
FROM {database_name}.feature_engineered_cleaned
GROUP BY source_dataset
ORDER BY row_count DESC
""")

rows_per_source

,source_dataset,row_count
0,TON_IoT,16023652
1,CIC-IDS2017,2827761
2,UNSW-NB15,2531675


### Label distribution in the merged dataset

In [57]:
label_dist = read_sql(f"""
SELECT
  label,
  COUNT(*) AS row_count
FROM {database_name}.feature_engineered_cleaned
GROUP BY label
ORDER BY label
""")

label_dist

,label,row_count
0,0,4852041
1,1,16531047


## Attack category distribution in the merged dataset

In [58]:
attack_category_dist = read_sql(f"""
SELECT
  COALESCE(NULLIF(trim(attack_category), ''), 'UNKNOWN') AS attack_category,
  COUNT(*) AS row_count
FROM {database_name}.feature_engineered_cleaned
GROUP BY 1
ORDER BY row_count DESC
""")

attack_category_dist

,attack_category,row_count
0,DoS/DDoS,9678799
1,Normal,4852041
2,Web Attack,2095484
3,Reconnaissance,1862138
4,Brute Force,1643527
5,Backdoor,510444
6,Injection,451709
7,Generic Malware,215651
8,Exploits,46032
9,Fuzzing,24226


## Label distribution by source dataset

In [59]:
label_dist_by_source = read_sql(f"""
SELECT
  source_dataset,
  label,
  COUNT(*) AS row_count
FROM {database_name}.feature_engineered_cleaned
GROUP BY source_dataset, label
ORDER BY source_dataset, label
""")

label_dist_by_source

,source_dataset,label,row_count
0,CIC-IDS2017,0,2271205
1,CIC-IDS2017,1,556556
2,TON_IoT,0,370348
3,TON_IoT,1,15653304
4,UNSW-NB15,0,2210488
5,UNSW-NB15,1,321187


## Attack category distribution by source dataset

In [60]:
attack_cat_by_source = read_sql(f"""
SELECT
  source_dataset,
  COALESCE(NULLIF(trim(attack_category), ''), 'UNKNOWN') AS attack_category,
  COUNT(*) AS row_count
FROM {database_name}.feature_engineered_cleaned
GROUP BY source_dataset, 2
ORDER BY source_dataset, row_count DESC
""")

attack_cat_by_source

,source_dataset,attack_category,row_count
0,CIC-IDS2017,Normal,2271205
1,CIC-IDS2017,DoS/DDoS,379737
2,CIC-IDS2017,Reconnaissance,158804
3,CIC-IDS2017,Brute Force,13832
4,CIC-IDS2017,Web Attack,2159
5,CIC-IDS2017,Botnet,1956
6,CIC-IDS2017,Infiltration,36
7,CIC-IDS2017,Injection,21
8,CIC-IDS2017,Exploits,11
9,TON_IoT,DoS/DDoS,9282720


## Sampling Before Stratified Dataset Split

The merged dataset contains tens of millions of network flow records, which might be unnecessary and inefficient for training a tree-based classification model in the context of this assignment. To reduce computational cost while preserving the statistical properties of the data, we perform a controlled sampling step before applying the stratified train/validation/test/production split.

The sampling is performed prior to splitting and is stratified by `(source_dataset, label)`, ensuring that:
- The relative contribution of each source dataset (TON_IoT, CIC-IDS2017, UNSW-NB15) is preserved.
- The benign/malicious label distribution within each dataset is maintained.
- Minority attack classes are not inadvertently removed.

In [61]:
# target total number of rows
TARGET_TOTAL_ROWS = 500_000

In [62]:
# drop sampled table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.merged_sampled_v2")

# delete S3 data directory to avoid HIVE_PATH_ALREADY_EXISTS error
import subprocess
subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/merged_sampled_v2/", "--recursive"], check=False)

# create a sampled version of the merged dataset before splitting
exec_ddl(f"""
CREATE TABLE {database_name}.merged_sampled_v2
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/merged_sampled_v2/',
  parquet_compression = 'SNAPPY'
) AS
WITH base AS (
  SELECT *
  FROM {database_name}.feature_engineered_cleaned
),

-- total rows per dataset
ds_counts AS (
  SELECT source_dataset, COUNT(*) AS ds_rows
  FROM base
  GROUP BY source_dataset
),

-- total rows per dataset + label
group_counts AS (
  SELECT source_dataset, label, COUNT(*) AS group_rows
  FROM base
  GROUP BY source_dataset, label
),

-- allocate target rows per dataset proportional to original size
ds_targets AS (
  SELECT
    source_dataset,
    CAST({TARGET_TOTAL_ROWS} AS DOUBLE)
      * (CAST(ds_rows AS DOUBLE) / SUM(CAST(ds_rows AS DOUBLE)) OVER ())
      AS ds_target_rows
  FROM ds_counts
),

-- allocate dataset targets down to label groups proportionally
group_targets AS (
  SELECT
    g.source_dataset,
    g.label,
    g.group_rows,
    d.ds_rows,
    t.ds_target_rows
      * (CAST(g.group_rows AS DOUBLE) / CAST(d.ds_rows AS DOUBLE))
      AS group_target_rows
  FROM group_counts g
  JOIN ds_counts d
    ON g.source_dataset = d.source_dataset
  JOIN ds_targets t
    ON g.source_dataset = t.source_dataset
),

-- compute sampling rate per (source_dataset, label)
rates AS (
  SELECT
    source_dataset,
    label,
    LEAST(1.0, group_target_rows / CAST(group_rows AS DOUBLE)) AS sample_rate
  FROM group_targets
)

SELECT b.*
FROM base b
JOIN rates r
  ON b.source_dataset = r.source_dataset
 AND b.label = r.label
WHERE rand() < r.sample_rate
""")

delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/merged_sampled_v2/20260210_021045_00052_e4wsk_d3d2a6e9-ffb5-43dc-b806-db0be17dd240
delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/merged_sampled_v2/20260210_021045_00052_e4wsk_b3337d1d-aedb-4772-8549-a097799c5407
delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/merged_sampled_v2/20260210_021045_00052_e4wsk_ca113b57-26b5-4055-8ad7-fc955b87a0f0
delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/merged_sampled_v2/20260210_021045_00052_e4wsk_5baa843a-0e5f-4b77-895a-ef56207d1c29
delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/merged_sampled_v2/20260210_021045_00052_e4wsk_394e7884-b398-4893-8554-b3f0a02a5a95


### Validate data source distribution

In [63]:
read_sql(f"""
SELECT source_dataset, COUNT(*) AS rows
FROM {database_name}.merged_sampled_v2
GROUP BY source_dataset
ORDER BY rows DESC
""")

,source_dataset,rows
0,TON_IoT,374989
1,CIC-IDS2017,66517
2,UNSW-NB15,59249


### Validate label distribution per dataset

In [64]:
read_sql(f"""
SELECT source_dataset, label, COUNT(*) AS rows
FROM {database_name}.merged_sampled_v2
GROUP BY source_dataset, label
ORDER BY source_dataset, label
""")

,source_dataset,label,rows
0,CIC-IDS2017,0,53410
1,CIC-IDS2017,1,13107
2,TON_IoT,0,8719
3,TON_IoT,1,366270
4,UNSW-NB15,0,51826
5,UNSW-NB15,1,7423


### Validate attack category distribution per dataset

In [65]:
read_sql(f"""
SELECT
  source_dataset,
  COALESCE(NULLIF(trim(attack_category), ''), 'UNKNOWN') AS attack_category,
  COUNT(*) AS row_count
FROM {database_name}.merged_sampled_v2
GROUP BY source_dataset, 2
ORDER BY source_dataset, row_count DESC
""")

,source_dataset,attack_category,row_count
0,CIC-IDS2017,Normal,53410
1,CIC-IDS2017,DoS/DDoS,8962
2,CIC-IDS2017,Reconnaissance,3709
3,CIC-IDS2017,Brute Force,340
4,CIC-IDS2017,Web Attack,53
5,CIC-IDS2017,Botnet,43
6,TON_IoT,DoS/DDoS,216763
7,TON_IoT,Web Attack,49148
8,TON_IoT,Reconnaissance,39590
9,TON_IoT,Brute Force,38258


## Stratified Dataset Split 
The dataset is partitioned into **train (40%)**, **validation (10%)**, **test (10%)**, and **production (40%)** categories. To handle the significant class and source imbalance (where the **TON_IoT** dataset is nearly 10x larger than others) we employ a **Stratified Splitting** strategy. Instead of a global split, we use the **`NTILE(100)`** window function partitioned by both `source_dataset` and `label`. This ensures that the 100 buckets are calculated independently for every unique combination of data source and class. As a result, each final split (e.g., the 'test' set) is guaranteed to contain exactly 10% of the samples from each specific dataset and each specific label.

### Determinism and Proportional Representation
By ordering the data within each partition by core flow features (such as `duration` and packet counts), the split remains fully deterministic and reproducible. This approach prevents any single dataset from disproportionately influencing a specific split and ensures that the model is validated and tested against a representative cross-section of all three telemetry sources.

In [66]:
# drop the split table if it already exists
exec_ddl(f"DROP TABLE IF EXISTS {database_name}.split_v2")

# delete S3 data directory to avoid HIVE_PATH_ALREADY_EXISTS error
subprocess.run(["aws", "s3", "rm", f"s3://{results_bucket}/aai540/processed/split_v2/", "--recursive"], check=False)

# create a new table with stratified splits
exec_ddl(f"""
CREATE TABLE {database_name}.split_v2
WITH (
  format = 'PARQUET',
  external_location = 's3://{results_bucket}/aai540/processed/split_v2/',
  parquet_compression = 'SNAPPY'
) AS

-- Using PARTITION BY ensures the 100 buckets are created for EACH dataset/label combo
WITH numbered_data AS (
  SELECT 
    *,
    NTILE(100) OVER (
        PARTITION BY source_dataset, label 
        ORDER BY 
            duration, 
            pkt_total, 
            bytes_total, 
            pkt_fwd, 
            pkt_bwd, 
            bytes_fwd, 
            bytes_bwd
    ) AS split_bucket
  FROM {database_name}.merged_sampled_v2
)

SELECT
  *,
  CASE
    WHEN split_bucket <= 40 THEN 'train'  -- 40% of each dataset/label
    WHEN split_bucket <= 50 THEN 'val'    -- 10% of each dataset/label
    WHEN split_bucket <= 60 THEN 'test'   -- 10% of each dataset/label
    ELSE 'prod'                           -- 40% of each dataset/label
  END AS data_split
FROM numbered_data
""")

delete: s3://sagemaker-us-east-1-933747558592/aai540/processed/split_v2/20260210_021057_00034_n8d95_a7ac9fed-6d14-4d74-932c-5ff04d8f4ea6


### Verify splits

### Verify split_v2 has all engineered features

In [ ]:
# Check columns in split_v2 - should have all 12 features
sample = read_sql(f"""
SELECT *
FROM {database_name}.split_v2
LIMIT 1
""")

print("Columns in split_v2:")
print(list(sample.columns))
print(f"\nTotal columns: {len(sample.columns)}")

# Verify engineered features are present
engineered_features = ['pkt_rate', 'byte_rate', 'bytes_per_pkt', 'pkt_ratio', 'byte_ratio']
missing = [f for f in engineered_features if f not in sample.columns]
if missing:
    print(f"\nMISSING engineered features: {missing}")
else:
    print(f"\n✓ All 5 engineered features present: {engineered_features}")

Columns in split_v2:
['duration', 'pkt_total', 'bytes_total', 'pkt_fwd', 'pkt_bwd', 'bytes_fwd', 'bytes_bwd', 'label', 'original_attack_type', 'attack_category', 'source_dataset', 'pkt_rate', 'byte_rate', 'bytes_per_pkt', 'pkt_ratio', 'byte_ratio', 'split_bucket', 'data_split']

Total columns: 18

✓ All 5 engineered features present: ['pkt_rate', 'byte_rate', 'bytes_per_pkt', 'pkt_ratio', 'byte_ratio']


In [67]:
read_sql(f"""
SELECT data_split, COUNT(*) AS rows,
       ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS pct
FROM {database_name}.split_v2
GROUP BY data_split
ORDER BY data_split
""")


,data_split,rows,pct
0,prod,200250,39.99
1,test,50070,10.00
2,train,200365,40.01
3,val,50070,10.00


### Verify data source distribution

In [68]:
read_sql(f"""
SELECT data_split, source_dataset, COUNT(*) rows,
       ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY data_split), 2) pct_within_split
FROM {database_name}.split_v2
GROUP BY data_split, source_dataset
ORDER BY data_split, source_dataset;
""")

,data_split,source_dataset,rows,pct_within_split
0,prod,CIC-IDS2017,26600,13.28
1,prod,TON_IoT,149970,74.89
2,prod,UNSW-NB15,23680,11.83
3,test,CIC-IDS2017,6650,13.28
4,test,TON_IoT,37500,74.90
5,test,UNSW-NB15,5920,11.82
6,train,CIC-IDS2017,26617,13.28
7,train,TON_IoT,150019,74.87
8,train,UNSW-NB15,23729,11.84
9,val,CIC-IDS2017,6650,13.28


### Verify label distribution in the train split

In [69]:
read_sql(f"""
SELECT 
    label, 
    COUNT(*) as row_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percent_of_train
FROM {database_name}.split_v2
WHERE data_split = 'train'
GROUP BY label
ORDER BY label
""")

,label,row_count,percent_of_train
0,0,45615,22.77
1,1,154750,77.23


### Verify attack category distribution in the train split

In [70]:
read_sql(f"""
SELECT 
    attack_category, 
    COUNT(*) as row_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percent_of_train
FROM {database_name}.split_v2
WHERE data_split = 'train'
GROUP BY attack_category
ORDER BY row_count DESC
""")

,attack_category,row_count,percent_of_train
0,DoS/DDoS,110006,54.90
1,Normal,45615,22.77
2,Reconnaissance,26816,13.38
3,Backdoor,11413,5.70
4,Generic Malware,2497,1.25
5,Brute Force,2377,1.19
6,Web Attack,1249,0.62
7,Exploits,158,0.08
8,Injection,133,0.07
9,Fuzzing,88,0.04


In [71]:
read_sql(f"""
SELECT 
    attack_category, 
    COUNT(*) as row_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percent_of_train
FROM {database_name}.split_v2
WHERE data_split = 'train'
GROUP BY attack_category
ORDER BY row_count DESC
""")

,attack_category,row_count,percent_of_train
0,DoS/DDoS,110006,54.90
1,Normal,45615,22.77
2,Reconnaissance,26816,13.38
3,Backdoor,11413,5.70
4,Generic Malware,2497,1.25
5,Brute Force,2377,1.19
6,Web Attack,1249,0.62
7,Exploits,158,0.08
8,Injection,133,0.07
9,Fuzzing,88,0.04
